In [1]:
from array import array
import subprocess
import math
import ROOT
import os
import numpy as np

Welcome to JupyROOT 6.29/01


In [2]:
buildcuda = '/home/mdessole/Projects/ROOT/genvectorx/build_cuda/'

In [3]:
buildoneapi = '/home/mdessole/Projects/ROOT/genvectorx/build_oneapi/'

In [4]:
buildopensycl = '/home/mdessole/Projects/ROOT/genvectorx/build_opensycl/'

In [5]:
!ls /home/mdessole/Projects/ROOT/genvectorx/build_opensycl/testx

CMakeFiles  cmake_install.cmake  CoordsOpenSYCL  RVecSYCL


In [6]:
!ls /home/mdessole/Projects/ROOT/genvectorx/build_oneapi/testx

CMakeFiles  cmake_install.cmake  CoordsOneAPI  RVecSYCL


In [7]:
!echo $LD_LIBRARY_PATH

/opt/intel/oneapi/tbb/2021.10.0/env/../lib/intel64/gcc4.8:/opt/intel/oneapi/debugger/2023.2.0/gdb/intel64/lib:/opt/intel/oneapi/debugger/2023.2.0/libipt/intel64/lib:/opt/intel/oneapi/debugger/2023.2.0/dep/lib:/opt/intel/oneapi/compiler/2023.2.1/linux/lib:/opt/intel/oneapi/compiler/2023.2.1/linux/lib/x64:/opt/intel/oneapi/compiler/2023.2.1/linux/lib/oclfpga/host/linux64/lib:/opt/intel/oneapi/compiler/2023.2.1/linux/compiler/lib/intel64_lin:/home/mdessole/Projects/sycl-blas/sycl-blas_build/benchmark/googlebench-build/src:/opt/OpenBLAS/lib:/home/mdessole/Projects/sycl-blas/sycl-blas_build/lib:/home/mdessole/Projects/sycl-blas/sycl-blas/lib:/home/mdessole/Projects/OpenSYCL/OpenSYCL_install/lib:/usr/local/cuda-11.8/lib64:/home/mdessole/Projects/ROOT/root/lib


In [8]:
sizes = array('d',[1000, 10000, 100000,1000000,5000000,10000000, 50000000,100000000]) #10,100,1000,10000,100000,
n = len(sizes)

In [14]:
def run_benchmark(builddir, sizes, nruns = 1, btype = 'cpu'):
    timeavg = array( 'd' )
    timestd = array( 'd' )
    
    if btype == 'cpu':
        exe = os.path.join(builddir,'testx','RVec')
    elif btype == 'cuda':
        exe = os.path.join(builddir,'testx','RVecCUDA')
    elif btype == 'sycl':
        exe = os.path.join(builddir,'testx','RVecSYCL')
    #endif
    
    if (not os.path.exists(exe)):
        print("Executable", exe, "not found!")
        return timeavg, timestd
    
    for i, N in enumerate(sizes):
        time = np.zeros(nruns)
        for r in range(nruns):
            result = subprocess.run([exe, str(int(N)) ], stdout=subprocess.PIPE).stdout.decode('utf-8')
            result = result.replace("\r","").replace("\n","")
            print(result)
            i = result.find("time")
            j = result.find("(s)")
            time[r] = float(result[i+5:j])
        #endfor
        timeavg.append(np.mean(time))
        timestd.append(np.std(time)) 
    #endfor
    return timeavg, timestd

In [12]:
timecpu, stdcpu = run_benchmark(buildoneapi, sizes, nruns = 3, btype = 'cpu')

In [ ]:
stdcpu

In [ ]:
timecpu

In [17]:
timeoneapi, stdoneapi = run_benchmark(buildoneapi, sizes, nruns = 3, btype = 'sycl')

sycl::queue check - selected device:NVIDIA GeForce RTX 3060sycl time 0.000768 (s)
sycl::queue check - selected device:NVIDIA GeForce RTX 3060sycl time 0.000672 (s)
sycl::queue check - selected device:NVIDIA GeForce RTX 3060sycl time 0.000676 (s)
sycl::queue check - selected device:NVIDIA GeForce RTX 3060sycl time 0.001054 (s)
sycl::queue check - selected device:NVIDIA GeForce RTX 3060sycl time 0.00112 (s)
sycl::queue check - selected device:NVIDIA GeForce RTX 3060sycl time 0.001268 (s)
sycl::queue check - selected device:NVIDIA GeForce RTX 3060sycl time 0.005929 (s)
sycl::queue check - selected device:NVIDIA GeForce RTX 3060sycl time 0.006932 (s)
sycl::queue check - selected device:NVIDIA GeForce RTX 3060sycl time 0.006022 (s)
sycl::queue check - selected device:NVIDIA GeForce RTX 3060sycl time 0.050504 (s)
sycl::queue check - selected device:NVIDIA GeForce RTX 3060sycl time 0.050833 (s)
sycl::queue check - selected device:NVIDIA GeForce RTX 3060sycl time 0.050318 (s)
sycl::queue check

In [18]:
timeoneapi

array('d', [0.0007053333333333333, 0.0011473333333333333, 0.006294333333333333, 0.05055166666666667, 0.24816833333333332, 0.4739933333333333, 2.241353333333333, 4.4444799999999995])

In [19]:
stdoneapi

array('d', [4.4342104395509066e-05, 8.947749561885505e-05, 0.00045249407608154255, 0.00021293243581526643, 0.0020442707474522207, 0.0014510066237692402, 0.0207944709531698, 0.0133256469511487])

In [21]:
timeopensycl, stdopensycl = run_benchmark(buildopensycl, sizes,  nruns = 3, btype = 'sycl')

[hipSYCL Warning] OpenCL device Intel(R) FPGA Emulation Device does not support Intel USM extensions; falling back to fine-grained system SVM. USM pointer info queries have limited support.[hipSYCL Warning] OpenCL device NVIDIA GeForce RTX 3060 does not have a valid USM provider. Memory allocations are not possible on that device.sycl::queue check - selected device:NVIDIA GeForce RTX 3060sycl time 0.001059 (s)
[hipSYCL Warning] OpenCL device Intel(R) FPGA Emulation Device does not support Intel USM extensions; falling back to fine-grained system SVM. USM pointer info queries have limited support.[hipSYCL Warning] OpenCL device NVIDIA GeForce RTX 3060 does not have a valid USM provider. Memory allocations are not possible on that device.sycl::queue check - selected device:NVIDIA GeForce RTX 3060sycl time 0.001082 (s)
[hipSYCL Warning] OpenCL device Intel(R) FPGA Emulation Device does not support Intel USM extensions; falling back to fine-grained system SVM. USM pointer info queries have

[hipSYCL Warning] OpenCL device Intel(R) FPGA Emulation Device does not support Intel USM extensions; falling back to fine-grained system SVM. USM pointer info queries have limited support.[hipSYCL Warning] OpenCL device NVIDIA GeForce RTX 3060 does not have a valid USM provider. Memory allocations are not possible on that device.sycl::queue check - selected device:NVIDIA GeForce RTX 3060sycl time 0.739556 (s)
[hipSYCL Warning] OpenCL device Intel(R) FPGA Emulation Device does not support Intel USM extensions; falling back to fine-grained system SVM. USM pointer info queries have limited support.[hipSYCL Warning] OpenCL device NVIDIA GeForce RTX 3060 does not have a valid USM provider. Memory allocations are not possible on that device.sycl::queue check - selected device:NVIDIA GeForce RTX 3060sycl time 0.73637 (s)
[hipSYCL Warning] OpenCL device Intel(R) FPGA Emulation Device does not support Intel USM extensions; falling back to fine-grained system SVM. USM pointer info queries have 

In [ ]:
stdopensycl 

In [24]:
timecuda, stdcuda = run_benchmark(buildcuda, sizes, nruns = 3, btype = 'cuda')

cuda time 0.071051 (s)
cuda time 0.062551 (s)
cuda time 0.085325 (s)
cuda time 0.06265 (s)
cuda time 0.078449 (s)
cuda time 0.063287 (s)
cuda time 0.079951 (s)
cuda time 0.064985 (s)
cuda time 0.065726 (s)
cuda time 0.079645 (s)
cuda time 0.080007 (s)
cuda time 0.081556 (s)
cuda time 0.124116 (s)
cuda time 0.131057 (s)
cuda time 0.126976 (s)
cuda time 0.181688 (s)
cuda time 0.177855 (s)
cuda time 0.176971 (s)
cuda time 0.637614 (s)
cuda time 0.62171 (s)
cuda time 0.620455 (s)
cuda time 1.15841 (s)
cuda time 1.17307 (s)
cuda time 1.15798 (s)


In [ ]:
timecuda

In [25]:
speedupopensycl = array('d', [timecpu[i]/timeopensycl[i] for i in range(n)])
speeduponeapi   = array('d', [timecpu[i]/timeoneapi[i] for i in range(n)])
speedupcuda     = array('d', [timecpu[i]/timecuda[i] for i in range(n)])

In [ ]:
val = list(sizes)
sizes_gb = [v*8*4/1e9 for v in val]
sizes_gb = array('d', sizes_gb)
sizes_gb

In [ ]:
p = ROOT.TGraph(len(sizes), sizes, speedupopensycl)
g = ROOT.TGraph(len(sizes), sizes, speeduponeapi)
q = ROOT.TGraph(len(sizes), sizes, speedupcuda)


c4 = ROOT.TCanvas()
g.SetMarkerStyle(7)
g.SetLineColor(ROOT.kBlue)
p.SetMarkerStyle(5)
p.SetLineColor(ROOT.kRed)
q.SetMarkerStyle(3)
q.SetLineColor(ROOT.kGreen)


q.SetTitle("Speedup wrt RVec")
q.GetXaxis().SetTitle( 'Input size (# doubles)' )
q.GetYaxis().SetTitle( 'Speedup' )

p.GetYaxis().SetRangeUser(0,6)

leg = ROOT.TLegend(0.7, 0.75, 0.30, 0.87)
leg.AddEntry(p,"OpenSYCL","L")
leg.AddEntry(g,"OneAPI","L")
leg.AddEntry(q,"CUDA","L")


c4.SetLogx();

q.Draw()
g.Draw('LP')
p.Draw('LP')
leg.Draw()
c4.Draw()

c4.SaveAs("speedup.png")

In [ ]:
p = ROOT.TGraphErrors(len(sizes), sizes, timeopensycl, stdopensycl)
g = ROOT.TGraphErrors(len(sizes), sizes, timeoneapi, stdoneapi)
q = ROOT.TGraphErrors(len(sizes), sizes, timecuda, stdcuda)
r = ROOT.TGraphErrors(len(sizes), sizes, timecpu, stdcpu)


c4 = ROOT.TCanvas()
g.SetLineColor(ROOT.kBlue)
p.SetLineColor(ROOT.kRed)
q.SetLineColor(ROOT.kGreen)


r.SetTitle("Execution times")
r.GetXaxis().SetTitle( 'Input size (# doubles)' )
r.GetYaxis().SetTitle( 'Time (s)' )

p.GetYaxis().SetRangeUser(0,6)

leg = ROOT.TLegend(0.7, 0.75, 0.30, 0.87)
leg.AddEntry(r,"RVec","L")
leg.AddEntry(p,"OpenSYCL","L")
leg.AddEntry(g,"OneAPI","L")
leg.AddEntry(q,"CUDA","L")

c4.SetLogy();
c4.SetLogx();

r.Draw()
q.Draw('LP')
g.Draw('LP')
p.Draw('LP')
leg.Draw()
c4.Draw()

c4.SaveAs("exectimes.png")